# IMPORT LIBRARIES

In [1]:
OPENSLIDE_PATH = r'E:/openslide-win64-20231011/openslide-win64-20231011/bin'

import os
if hasattr(os, 'add_dll_directory'):
    # Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [2]:
import os
import cv2
import time
import random
import shutil
import fnmatch
import numpy as np
import pandas as pd

# OPEN ANNOTATIONS

In [3]:
df = pd.read_csv('D:/annotation/NB.csv')

In [4]:
images_df = [os.path.join("G:/NB/", df.iloc[i]["images"]) for i in range(len(df))]
patches_df = [os.path.join("G:/cut-image/NB/", df.iloc[i]["folders"]) for i in range(len(df))]
annos_df = [os.path.join("D:/annotation/NB/", df.iloc[i]["annotations"]) for i in range(len(df))]

In [5]:
# for i in patches_df:
#     os.makedirs(i)

# CUT IMAGES

In [6]:
df = pd.DataFrame({'images': images_df,
                    'anno': annos_df,
                    'folder': patches_df
                  })

In [7]:
def read_anno(anno_path):
    raw_data = pd.read_json(anno_path)
    anno = raw_data.annotation
    els = [x['elements'] for x in anno if (len(x['elements']) > 0)]
    pts = []
    labels = []
    rgb = []
    for e in els:
        for p in e:
            count = 0
            if (p.get('label') != None and 
                (p.get('label').get('value') == 'Ung thư biểu mô TKNT tế bào lớn' or
                p.get('label').get('value') == 'Ung thư biểu mô, không thể định loại')):
                pts.append(p.get("points"))
                t_label = p.get('label').get('value')
                # if (t_label == 'Ung thư biểu mô tuyến - solid' or 
                #     t_label == 'Ung thư biểu mô KTBN, thiên về UTBM tuyến' or
                #     t_label == 'Ung thư biểu mô tuyến - Acinar' or
                #     t_label == 'Ung thư biểu mô tuyến - Lepidic' or 
                #     t_label == 'Ung thư biểu mô tuyến - nhú' or
                #     t_label == 'Ung thư biểu mô tuyến - vi nhú' or
                #     t_label == ' Ung thư biểu mô tuyến - típ ruột' or
                #     t_label == 'Ung thư biểu mô tuyến - keo' or
                #     t_label == 'Ung thư biểu mô tuyến - bào thai' or
                #     t_label == 'Ung thư biểu mô tuyến - nhầy'):
                #     label = 'Ung thư biểu mô tuyến'
                #     color = (255, 255, 204)
                # elif (t_label == 'Ung thư biểu mô tế bào vảy' or 
                #     t_label == 'Ung thư biểu mô KTBN thiên về UTBM vảy'):
                #     label = 'Ung thư biểu mô vảy'
                #     color = (204, 255, 229)
                if (t_label == 'Ung thư biểu mô TKNT tế bào lớn'):
                    label = 'Ung thư biểu mô TKNT tế bào lớn'
                    color = (204, 255, 255)
                    count += 1
                elif (t_label == 'Ung thư biểu mô, không thể định loại'):
                    label = 'Ung thư biểu mô, không thể định loại'
                    color = (204, 204, 204)
                    count += 1
                # elif (t_label == 'Ung thư biểu mô tế bào nhỏ'):
                #     label = 'Ung thư biểu mô tế bào nhỏ'
                #     color = (255, 204, 204)
                # else:
                #     label = 'Bình thường'
                #     color = (255, 229, 204)
            if (count != 0):
                labels.append(label)
                rgb.append(color)
            else:
                return 0
    polys = []
    for p in pts:
        poly = []
        for x in p:
            x_coord = round(x[0])
            y_coord = round(x[1])
            poly.append([x_coord, y_coord])
        polygon = np.array(poly)
        polys.append(polygon)
    ill = pd.DataFrame(
    {'coords': polys,
     'labels': labels,
     'color': rgb
    })
    return ill

In [8]:
def extract_patch(ill, size):
    tile = []
    for i in range(len(ill)):
        p = ill.loc[i, "coords"]
        x_min = size.dimensions[0]
        y_min = size.dimensions[1]
        x_max = 0
        y_max = 0
        for x in p:
            x_min = min(x_min, x[0])
            y_min = min(y_min, x[1])
            x_max = max(x_max, x[0])
            y_max = max(y_max, x[1])
        tile.append([x_min, y_min, x_max, y_max])
    return tile

In [9]:
def get_patch(os_obj, tl_pixel, patch_shape):
    return os_obj.read_region(tl_pixel, 0, patch_shape).convert("RGB")

In [10]:
PATCH_SHAPE = [512, 512]

for id in range(len(df)):
    index = id + 492
    t1 = time.time()
    OS_OBJ = openslide.open_slide(df.iloc[index]['images'])
    ill_df = read_anno(df.iloc[index]['anno'])
    if (isinstance(ill_df, int) == True):
        continue
    foldername = df.iloc[index]['folder']
    patch = extract_patch(ill_df, OS_OBJ)
    print(foldername)
    print("demo dimensions: ", OS_OBJ.dimensions) #W, H -> x, y
    tot_mask = np.zeros((OS_OBJ.dimensions[1], OS_OBJ.dimensions[0], 3), dtype = "uint8")
    
    for i in patch:
        for j in range(len(ill_df)):
            coords = ill_df.iloc[j]['coords']
            color = ill_df.iloc[j]['color']
            tot_mask = cv2.fillPoly(tot_mask, [coords], color)

        LEFT_PATCH_TL_PIXEL = [round(i[0]),round(i[1])]
        RIGHT_PATCH_TL_PIXEL = [round(i[2]),round(i[3])]

        if ((OS_OBJ.dimensions[0]<(RIGHT_PATCH_TL_PIXEL[0])) or
            (OS_OBJ.dimensions[1]<(RIGHT_PATCH_TL_PIXEL[1]))):
                LEFT_PATCH_TL_PIXEL = (0,0)

        y = round(i[1])
        
        while (y < (round(i[3]))):
            x = round(i[0])
            while (x < (round(i[2]))):
                START_PIXEL = [x, y]
                PATCH = get_patch(OS_OBJ, START_PIXEL, PATCH_SHAPE)
                patch_id = f"{'x_'}{x}{'_y_'}{y}{'_'}{foldername[16:]}" + ".png"
                path_image=foldername + '/' + patch_id
                
                
                mask = np.zeros((512, 512, 3))
                mask = tot_mask[y:y + 512, x:x + 512]
                check = (mask > 0).sum()
                mask_id = f"{'x_'}{x}{'_y_'}{y}{'_'}{foldername[16:]}{'mask'}" + ".png"
                path_mask=foldername + '/' + mask_id
                if (check > 0 and mask.shape[0] == 512 and mask.shape[1] == 512):
                    cv2.imwrite(path_mask, mask)
                    PATCH.save(path_image)
                x = x + 512
            y = y + 512

G:/cut-image/NB/AD7151
demo dimensions:  (76800, 111616)
G:/cut-image/NB/AC5798
demo dimensions:  (24576, 76800)


IndexError: single positional indexer is out-of-bounds

# SELECT RANDOM IMAGES

In [ ]:
# folders = os.listdir('G:/NB_w_anno/')
# BATCH_SIZE = 12

# for file_name in folders:
#     file_path = os.path.join('G:/NB_w_anno_random/', file_name)
#     os.makedirs(file_path)

# for f in folders:
    
#     print(f)
    
#     masks = fnmatch.filter(os.listdir('G:/NB_w_anno/' + f + '/'), '*mask.png')
    
#     if (len(masks) >= BATCH_SIZE):
#         masks = random.sample(masks, 12)

#     for i in masks:
#         image = 'G:/NB_w_anno/' + f +'/' + i[:-8] + '.png'
#         mask = 'G:/NB_w_anno/' + f +'/' + i
#         path = 'G:/NB_w_anno_random/' + f
#         shutil.move(image, path)
#         shutil.move(mask, path)